# Import data
Given the shape of our data, the best option is probably to use a [`pandas.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) to work with the data we want to analyze. Pandas has a very useful function, [`pandas.read_excel`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html) to import data from an Excel file into a `pandas.DataFrame`.

Given the shape of our spreadsheet, we face (at least) three issues:
1. There are some rows, at the beginning of the spreadseet, that we do not want to read/use.
2. The shape of the Excel file is, basically, the opposite of what is the convention for `pandas`. The index (the time period) is usually in rows instead of columns.
3. Missing data is recorded by Eurostat using `:` while the proper way to record a missing observation is using a `NaN` record.
4. Some country names (Germany and Macedonia) are recorded poorly to be used (in a loop for example).
5. A key issue when using `pandas` classes is [indexing](https://pandas.pydata.org/pandas-docs/stable/indexing.html). Since we are dealing with (a set of) [time series](https://pandas.pydata.org/pandas-docs/stable/timeseries.html), we want the index to have the proper functionality. In particular, our data refers to a full calendar year quarter which can be represented using a [period](https://pandas.pydata.org/pandas-docs/stable/timeseries.html#periodindex-and-period-range). This sort of information cannot be read from the Excel file.

## Read the Excel file and transpose it
We want to:
- **Not** read the first rows of the spreadsheet: we can use the `skiprow` option of `pandas.read_excel`. 
- Correct for the fact that the shape of the Excel file is the opposite of what we need:
    - We can use the method [`DataFrame.T`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.T.html#pandas.DataFrame.T) to transpose the `pandas.DataFrame`.
    - We can set the country as index (using the `index_col` option of `pandas.read_excel`), **before** transposing, in order to have a cleaner transposition.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
MyFolder = '/content/drive/My Drive/Colab Notebooks/2021_2022_Programming_for_Statistical_Analysis/'
MyFile = MyFolder + 'Eurostat.xls' 

In [ ]:
import pandas as pd

UnemploymentRate = pd.read_excel(MyFile, skiprows=11, index_col=0)
UnemploymentRate = UnemploymentRate.T
UnemploymentRate

## Replace `:` for `NaN`

In [ ]:
import numpy as np

UnemploymentRate = UnemploymentRate.replace(':', np.NaN)
UnemploymentRate

## Shorten country names Germany and Macedonia
We can use the [`DataFrame.rename()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html#pandas.DataFrame.rename) method.

In [ ]:
UnemploymentRate = UnemploymentRate.rename(columns = {'Germany (until 1990 former territory of the FRG)':'Germany'})
UnemploymentRate = UnemploymentRate.rename(columns = {'Former Yugoslav Republic of Macedonia, the':'Macedonia'})
UnemploymentRate

We can also change the name of the list which stores the names of the countries:

In [ ]:
UnemploymentRate.columns.name = 'Countries'
UnemploymentRate

## Proper time series index
Although it *looks* correct, currently, our `pandas.DataFrame` instance `UnemploymentRate` does not know that is containing time series:

In [ ]:
UnemploymentRate.index

The index is just a list of strings, not a range of periods. We can define a range of periods by:

In [ ]:
dates = pd.period_range(start='2010Q2', end='2017Q4', freq='Q', name='Quarterly Frequency')
dates

In [ ]:
UnemploymentRate.index = dates
UnemploymentRate

# Plot selected time series
It's straight forward to use `pandas` to create data [visulizations](https://pandas.pydata.org/pandas-docs/stable/visualization.html).

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = [12.0,6.0] # Modifies the defult size of plots [6.0,4.0] inches

In [ ]:
UnemploymentRate['Switzerland'].plot(legend=True)
# UnemploymentRate['Macedonia'].plot(legend=True)
UnemploymentRate['Germany'].plot(legend=True)
# UnemploymentRate['Sweden'].plot(legend=True)
# UnemploymentRate['Spain'].plot(legend=True)
# UnemploymentRate['Austria'].plot(legend=True)
# UnemploymentRate['Greece'].plot(legend=True)
plt.ylabel('Unemployment Rate (%)')
plt.show()